In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
from keras.utils import to_categorical
import random
from tensorflow import set_random_seed
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

[nltk_data] Downloading package wordnet to C:\Users\sreekar
[nltk_data]     chidurala\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


In [4]:
#!pip install matplotlib
#!pip3 install nltk
#!pip install sklearn
#!pip install scipy
#!pip3 install pandas


In [5]:
import tensorflow as tf
#!pip install matplotlib
from keras.utils import to_categorical

In [24]:
#!pip install opencv-python

In [6]:
#!pip install tqdm

In [2]:
train1=pd.read_csv("train.tsv",sep="\t")

In [3]:
test1=pd.read_csv("test.tsv",sep="\t")

In [36]:
import pandas as pd
train1.head()
train1.shape , test1.shape

((156060, 4), (66292, 3))

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
stop_words=nltk.corpus.stopwords.words('english')
stop_words=list(np.setdiff1d(stop_words,['not']))
junk_words=[]
len(stop_words)
stop_words.extend(junk_words)
len(stop_words)

178

In [7]:
docs=train1['Phrase'].fillna('').str.lower().str.replace('[^a-z ]','')
docs.head()

0    a series of escapades demonstrating the adage ...
1    a series of escapades demonstrating the adage ...
2                                             a series
3                                                    a
4                                               series
Name: Phrase, dtype: object

In [39]:
def clean_sentences(df):
    reviews = []

    for i in tqdm(df['Phrase']):
        
        #remove html content
        review_text = BeautifulSoup(i).get_text()
        
        #remove non-alphabetic characters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
        #tokenize the sentences
        words = word_tokenize(review_text.lower())
    
        #lemmatize each word to its lemma
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
    
        reviews.append(lemma_words)

    return(reviews)

stemmer=nltk.PorterStemmer()
def clean_text(row_text):
    #print(type(row_text))
    row_words=row_text.split(' ')
    #print(row_words)
    row_words= [stemmer.stem(word) for word in row_words if word not in stop_words]
    words = word_tokenize(row_words)
    #lemmatize each word to its lemma
    lem_words = [lemmatizer.lemmatize(i) for i in words]
    #print(row_words)
    #print('----')
    return ' '.join(lem_words)

docs_clean=docs.apply(lambda v: clean_text(v))

In [37]:
docs_clean=clean_sentences(train1)

100%|██████████| 156060/156060 [00:46<00:00, 3329.33it/s]


In [10]:
#docs_clean

In [40]:
test_sentences = clean_sentences(test1)

100%|██████████| 66292/66292 [00:18<00:00, 3542.15it/s]


In [44]:
target=train1.Sentiment.values
y_target=to_categorical(target)
num_classes=y_target.shape[1]

In [45]:
X_train,X_val,y_train,y_val=train_test_split(docs_clean,y_target,test_size=0.2,stratify=y_target)

In [46]:
unique_words = set()
len_max = 0

for sent in tqdm(X_train):
    
    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max = len(sent)
        
#length of the list of unique_words gives the no of unique words
print(len(list(unique_words)))
print(len_max)

100%|██████████| 124848/124848 [00:00<00:00, 588522.08it/s]


13740
48


In [47]:
tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(test_sentences)

#padding done to equalize the lengths of all input reviews. LSTM networks needs all inputs to be same length.
#Therefore reviews lesser than max length will be made equal using extra zeros at end. This is padding.
X_train = sequence.pad_sequences(X_train, maxlen=len_max)
X_val = sequence.pad_sequences(X_val, maxlen=len_max)
X_test = sequence.pad_sequences(X_test, maxlen=len_max)
print(X_train.shape,X_val.shape,X_test.shape)

(124848, 48) (31212, 48) (66292, 48)


In [48]:
early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor='val_acc', patience = 2)
callback = [early_stopping]

#Model using Keras LSTM
model=Sequential()
model.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=True))
model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.005),metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 48, 300)           4122000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 48, 128)           219648    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 505       
Total params: 4,398,061
Trainable params: 4,398,061
Non-trainable params: 0
_________________________________________________________________


In [17]:
history=model.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=6, batch_size=256, verbose=1, callbacks=callback)

Instructions for updating:
Use tf.cast instead.
Train on 124848 samples, validate on 31212 samples
Epoch 1/6


 51456/124848 [===========>..................] - ETA: 49:30:56 - loss: 1.6118 - acc: 0.199 - ETA: 24:44:24 - loss: 1.5865 - acc: 0.300 - ETA: 16:28:51 - loss: 1.5133 - acc: 0.373 - ETA: 12:21:07 - loss: 1.4812 - acc: 0.408 - ETA: 9:52:26 - loss: 1.4234 - acc: 0.439 - ETA: 8:13:19 - loss: 1.4089 - acc: 0.44 - ETA: 7:02:32 - loss: 1.3790 - acc: 0.46 - ETA: 6:09:26 - loss: 1.3762 - acc: 0.46 - ETA: 5:28:08 - loss: 1.3643 - acc: 0.46 - ETA: 4:55:04 - loss: 1.3566 - acc: 0.46 - ETA: 4:28:03 - loss: 1.3406 - acc: 0.47 - ETA: 4:05:31 - loss: 1.3365 - acc: 0.47 - ETA: 3:46:26 - loss: 1.3299 - acc: 0.48 - ETA: 3:30:06 - loss: 1.3265 - acc: 0.48 - ETA: 3:15:56 - loss: 1.3193 - acc: 0.48 - ETA: 3:03:32 - loss: 1.3160 - acc: 0.48 - ETA: 2:52:36 - loss: 1.3146 - acc: 0.48 - ETA: 2:42:53 - loss: 1.3066 - acc: 0.48 - ETA: 2:34:10 - loss: 1.3036 - acc: 0.48 - ETA: 2:26:21 - loss: 1.2994 - acc: 0.49 - ETA: 2:19:16 - loss: 1.2971 - acc: 0.49 - ETA: 2:12:49 - loss: 1.2911 - acc: 0.49 - ETA: 2:06:56 - los

 52224/124848 [===========>..................] - ETA: 3:50 - loss: 0.7856 - acc: 0.703 - ETA: 3:51 - loss: 0.7716 - acc: 0.689 - ETA: 3:52 - loss: 0.7776 - acc: 0.695 - ETA: 3:52 - loss: 0.7509 - acc: 0.698 - ETA: 3:53 - loss: 0.7584 - acc: 0.689 - ETA: 3:53 - loss: 0.7698 - acc: 0.681 - ETA: 3:53 - loss: 0.7853 - acc: 0.680 - ETA: 3:53 - loss: 0.7921 - acc: 0.675 - ETA: 3:51 - loss: 0.7871 - acc: 0.678 - ETA: 3:50 - loss: 0.7862 - acc: 0.676 - ETA: 3:50 - loss: 0.7875 - acc: 0.675 - ETA: 3:49 - loss: 0.7880 - acc: 0.674 - ETA: 3:49 - loss: 0.7842 - acc: 0.676 - ETA: 3:48 - loss: 0.7865 - acc: 0.676 - ETA: 3:48 - loss: 0.7908 - acc: 0.675 - ETA: 3:48 - loss: 0.7857 - acc: 0.676 - ETA: 3:47 - loss: 0.7898 - acc: 0.676 - ETA: 3:47 - loss: 0.7908 - acc: 0.676 - ETA: 3:46 - loss: 0.7884 - acc: 0.677 - ETA: 3:45 - loss: 0.7905 - acc: 0.674 - ETA: 3:45 - loss: 0.7876 - acc: 0.676 - ETA: 3:45 - loss: 0.7842 - acc: 0.678 - ETA: 3:44 - loss: 0.7857 - acc: 0.676 - ETA: 3:43 - loss: 0.7885 - acc:

124848/124848 [==============================] - ETA: 37s - loss: 0.8064 - acc: 0.67 - ETA: 36s - loss: 0.8068 - acc: 0.67 - ETA: 36s - loss: 0.8068 - acc: 0.67 - ETA: 35s - loss: 0.8067 - acc: 0.67 - ETA: 35s - loss: 0.8070 - acc: 0.66 - ETA: 34s - loss: 0.8070 - acc: 0.66 - ETA: 34s - loss: 0.8068 - acc: 0.67 - ETA: 33s - loss: 0.8067 - acc: 0.67 - ETA: 33s - loss: 0.8067 - acc: 0.67 - ETA: 32s - loss: 0.8066 - acc: 0.67 - ETA: 32s - loss: 0.8067 - acc: 0.67 - ETA: 31s - loss: 0.8067 - acc: 0.67 - ETA: 31s - loss: 0.8067 - acc: 0.67 - ETA: 30s - loss: 0.8066 - acc: 0.67 - ETA: 30s - loss: 0.8066 - acc: 0.67 - ETA: 29s - loss: 0.8067 - acc: 0.67 - ETA: 29s - loss: 0.8066 - acc: 0.67 - ETA: 29s - loss: 0.8068 - acc: 0.67 - ETA: 28s - loss: 0.8067 - acc: 0.67 - ETA: 28s - loss: 0.8069 - acc: 0.66 - ETA: 27s - loss: 0.8068 - acc: 0.66 - ETA: 27s - loss: 0.8070 - acc: 0.66 - ETA: 26s - loss: 0.8070 - acc: 0.66 - ETA: 26s - loss: 0.8070 - acc: 0.66 - ETA: 25s - loss: 0.8069 - acc: 0.66 - E

 52224/124848 [===========>..................] - ETA: 3:52 - loss: 0.6642 - acc: 0.738 - ETA: 3:52 - loss: 0.6756 - acc: 0.736 - ETA: 3:52 - loss: 0.6775 - acc: 0.737 - ETA: 3:58 - loss: 0.6946 - acc: 0.727 - ETA: 3:58 - loss: 0.7128 - acc: 0.716 - ETA: 3:57 - loss: 0.7079 - acc: 0.714 - ETA: 3:57 - loss: 0.7113 - acc: 0.713 - ETA: 4:00 - loss: 0.7074 - acc: 0.712 - ETA: 4:01 - loss: 0.7093 - acc: 0.715 - ETA: 4:00 - loss: 0.7143 - acc: 0.712 - ETA: 3:58 - loss: 0.7180 - acc: 0.713 - ETA: 3:57 - loss: 0.7151 - acc: 0.714 - ETA: 3:56 - loss: 0.7172 - acc: 0.712 - ETA: 3:56 - loss: 0.7229 - acc: 0.707 - ETA: 3:55 - loss: 0.7156 - acc: 0.712 - ETA: 3:54 - loss: 0.7168 - acc: 0.712 - ETA: 3:53 - loss: 0.7140 - acc: 0.713 - ETA: 3:52 - loss: 0.7113 - acc: 0.714 - ETA: 3:51 - loss: 0.7123 - acc: 0.714 - ETA: 3:50 - loss: 0.7135 - acc: 0.711 - ETA: 3:49 - loss: 0.7149 - acc: 0.710 - ETA: 3:48 - loss: 0.7144 - acc: 0.709 - ETA: 3:48 - loss: 0.7159 - acc: 0.708 - ETA: 3:47 - loss: 0.7160 - acc:

 52224/124848 [===========>..................] - ETA: 4:17 - loss: 0.6431 - acc: 0.722 - ETA: 4:25 - loss: 0.6687 - acc: 0.693 - ETA: 4:14 - loss: 0.6594 - acc: 0.701 - ETA: 4:09 - loss: 0.6530 - acc: 0.702 - ETA: 4:05 - loss: 0.6416 - acc: 0.715 - ETA: 4:03 - loss: 0.6562 - acc: 0.707 - ETA: 4:02 - loss: 0.6588 - acc: 0.712 - ETA: 4:02 - loss: 0.6538 - acc: 0.717 - ETA: 4:00 - loss: 0.6512 - acc: 0.720 - ETA: 3:58 - loss: 0.6517 - acc: 0.722 - ETA: 3:57 - loss: 0.6569 - acc: 0.718 - ETA: 3:56 - loss: 0.6576 - acc: 0.719 - ETA: 3:55 - loss: 0.6516 - acc: 0.724 - ETA: 3:54 - loss: 0.6552 - acc: 0.722 - ETA: 3:53 - loss: 0.6572 - acc: 0.722 - ETA: 3:52 - loss: 0.6526 - acc: 0.723 - ETA: 3:51 - loss: 0.6542 - acc: 0.724 - ETA: 3:51 - loss: 0.6534 - acc: 0.723 - ETA: 3:50 - loss: 0.6517 - acc: 0.723 - ETA: 3:49 - loss: 0.6566 - acc: 0.721 - ETA: 3:49 - loss: 0.6558 - acc: 0.721 - ETA: 3:48 - loss: 0.6579 - acc: 0.720 - ETA: 3:47 - loss: 0.6550 - acc: 0.722 - ETA: 3:47 - loss: 0.6557 - acc:

124848/124848 [==============================] - ETA: 37s - loss: 0.6907 - acc: 0.70 - ETA: 37s - loss: 0.6910 - acc: 0.70 - ETA: 36s - loss: 0.6910 - acc: 0.70 - ETA: 36s - loss: 0.6912 - acc: 0.70 - ETA: 35s - loss: 0.6912 - acc: 0.70 - ETA: 35s - loss: 0.6914 - acc: 0.70 - ETA: 34s - loss: 0.6917 - acc: 0.70 - ETA: 34s - loss: 0.6915 - acc: 0.70 - ETA: 33s - loss: 0.6915 - acc: 0.70 - ETA: 33s - loss: 0.6914 - acc: 0.70 - ETA: 32s - loss: 0.6915 - acc: 0.70 - ETA: 32s - loss: 0.6916 - acc: 0.70 - ETA: 31s - loss: 0.6918 - acc: 0.70 - ETA: 31s - loss: 0.6917 - acc: 0.70 - ETA: 30s - loss: 0.6918 - acc: 0.70 - ETA: 30s - loss: 0.6918 - acc: 0.70 - ETA: 29s - loss: 0.6919 - acc: 0.70 - ETA: 29s - loss: 0.6920 - acc: 0.70 - ETA: 28s - loss: 0.6922 - acc: 0.70 - ETA: 28s - loss: 0.6922 - acc: 0.70 - ETA: 27s - loss: 0.6924 - acc: 0.70 - ETA: 27s - loss: 0.6926 - acc: 0.70 - ETA: 26s - loss: 0.6927 - acc: 0.70 - ETA: 26s - loss: 0.6929 - acc: 0.70 - ETA: 25s - loss: 0.6930 - acc: 0.70 - E

 52224/124848 [===========>..................] - ETA: 3:51 - loss: 0.6711 - acc: 0.730 - ETA: 3:53 - loss: 0.6505 - acc: 0.734 - ETA: 3:51 - loss: 0.6180 - acc: 0.751 - ETA: 3:51 - loss: 0.6147 - acc: 0.754 - ETA: 3:51 - loss: 0.6028 - acc: 0.759 - ETA: 3:51 - loss: 0.6032 - acc: 0.758 - ETA: 3:50 - loss: 0.6139 - acc: 0.748 - ETA: 3:51 - loss: 0.6171 - acc: 0.744 - ETA: 3:50 - loss: 0.6200 - acc: 0.740 - ETA: 3:49 - loss: 0.6180 - acc: 0.741 - ETA: 3:49 - loss: 0.6196 - acc: 0.742 - ETA: 3:49 - loss: 0.6278 - acc: 0.738 - ETA: 3:48 - loss: 0.6216 - acc: 0.741 - ETA: 3:47 - loss: 0.6215 - acc: 0.742 - ETA: 3:47 - loss: 0.6240 - acc: 0.741 - ETA: 3:46 - loss: 0.6254 - acc: 0.741 - ETA: 3:46 - loss: 0.6245 - acc: 0.742 - ETA: 3:45 - loss: 0.6258 - acc: 0.739 - ETA: 3:45 - loss: 0.6282 - acc: 0.737 - ETA: 3:45 - loss: 0.6266 - acc: 0.737 - ETA: 3:44 - loss: 0.6267 - acc: 0.737 - ETA: 3:44 - loss: 0.6278 - acc: 0.736 - ETA: 3:43 - loss: 0.6292 - acc: 0.737 - ETA: 3:43 - loss: 0.6309 - acc:

In [49]:
X_test

array([[   0,    0,    0, ...,  619,  819,  319],
       [   0,    0,    0, ...,  619,  819,  319],
       [   0,    0,    0, ...,    0,    0,   15],
       ...,
       [   0,    0,    0, ...,    2,  131, 5334],
       [   0,    0,    0, ...,    2,  131, 5334],
       [   0,    0,    0, ...,    0,  384, 1870]])

In [50]:
#make the predictions with trained model and submit the predictions.
y_pred=model.predict_classes(X_test)

In [51]:
len(y_pred)


66292

In [54]:
sub_file = pd.read_csv('sampleSubmission.csv',sep=',')
sub_file.Sentiment=y_pred
sub_file.to_csv('Submission1.csv',index=False)

In [53]:
sub_file.shape

(66292, 2)

In [26]:
testy=pd.read_csv("test.tsv",sep="\t")

In [27]:
testy.shape

(66292, 3)

In [28]:
train1.shape

(156060, 4)

In [43]:
train,test=train_test_split(docs_clean,test_size=.3,random_state=100)
train_y=train1.loc[train.index]['Sentiment']
test_y=train1.loc[test.index]['Sentiment']

C:\Users\sreekar chidurala\Anaconda3\envs\sreekarr\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [11]:
cv=CountVectorizer()
cv.fit(train)
train_x=cv.transform(train)
test_x=cv.transform(test)

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [13]:
adaboost_model=RandomForestClassifier(n_estimators=3,random_state=100)
adaboost_model.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=3, n_jobs=None,
            oob_score=False, random_state=100, verbose=0, warm_start=False)

In [14]:
pred_class=adaboost_model.predict(test_x)

In [15]:
from sklearn.metrics import accuracy_score,f1_score,roc_curve,auc

In [16]:
print(accuracy_score(test_y,pred_class))

0.6197189115297536
